**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
# import skvideo
# skvideo.setFFmpegPath("D:\Anaconda\Lib\site-packages\\ffmpeg")
import skvideo.io
import cv2
import json

import tensorflow as tf
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function returns the action to perform given the current state.  𝜖  is essential there to force the exploration behaviour of the agent. Indeed, with this called  𝜖 -greedy policy, the agents performs a random action with probability  𝜖  that leads him to go to other states not dictated by its "optimal up to now" policy and maybe find out that there are other actions that are far more rewarding

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board is the grid where the agent will move around. Its cells are either 0.5 (a cheese), 0 (normal path) or -1 (a poison).

The position array tracks the movement of the agent on the board : the value of each cell where the agent passes is set to 1. It has the same shape as the board array but its boundaries are padded by -1 values ( left-right and up-down) to warn the agent (that has a visibility of two cells) that it reached the boundary of the environment

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        s = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # For testing, we use the learned policy and the the epsilon greedy, it is the exploitation phase
            a = agent.act(s, train=False) 
            old_s= s
            s, r, game_over = env.act(a)
            if r >0:
                win = win +r 
            if r < 0:
                lose = lose-r
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/10.0. Average score (2.0)
Win/lose count 15.5/10.0. Average score (3.75)
Win/lose count 7.0/18.0. Average score (-1.1666666666666667)
Win/lose count 8.5/11.0. Average score (-1.5)
Win/lose count 12.5/9.0. Average score (-0.5)
Win/lose count 9.0/11.0. Average score (-0.75)
Win/lose count 9.0/8.0. Average score (-0.5)
Win/lose count 11.5/17.0. Average score (-1.125)
Win/lose count 6.0/14.0. Average score (-1.8888888888888888)
Win/lose count 8.0/17.0. Average score (-2.6)
Win/lose count 11.5/15.0. Average score (-2.6818181818181817)
Win/lose count 8.5/25.0. Average score (-3.8333333333333335)
Win/lose count 9.0/10.0. Average score (-3.6153846153846154)
Win/lose count 11.5/12.0. Average score (-3.392857142857143)
Win/lose count 12.0/13.0. Average score (-3.2333333333333334)
Win/lose count 9.0/7.0. Average score (-2.90625)
Win/lose count 8.0/15.0. Average score (-3.1470588235294117)
Win/lose count 12.5/18.0. Average score (-3.2777777777777777)
Win/lose count 13.0/23.0. A

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have that : (using Markov property and a change of "time")
$$
\begin{split}
Q^{\pi}(s, a)    & = \mathbb{E}_{\pi}[\sum_{t=0}^{\infty} \gamma^{t} r(s_t, a_t) | s_0=s, a_0=a] \\
                 & = \mathbb{E}_{\pi}[r(s,a) + \sum_{t=1}^{\infty} \gamma^{t} r(s_t, a_t) | s_0=s, a_0=a]\\
                 & = r(s,a) + \gamma \mathbb{E}_{\pi}[ \sum_{t=1}^{\infty} \gamma^{t-1} r(s_{t}, a_{t}) | s_0=s, a_0=a]\\
                 & = r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \mathbb{E}_{\pi}[ \sum_{t=1}^{\infty} \gamma^{t-1} r(s_{t}, a_{t}) | s_1=s', a_1=\pi(s')]\\
                 & = r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \mathbb{E}_{\pi}[ \sum_{t'=0}^{\infty} \gamma^{t'} r(s_{t'}, a_{t'}) | s_{0'}=s', a_{0'}=\pi(s')]\\
                 & = r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \mathbb{E}_{a' \sim \pi}[ \sum_{t'=0}^{\infty} \gamma^{t'} r(s_{t'}, a_{t'}) | s_{0'}=s', a_{0'}=a']\\
                 & = r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) Q^{\pi}(s', a') \\
                 & = r(s,a) + \gamma \mathbb{E}_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s', a'))] \\
                 & = \mathbb{E}_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s', a'))]
\end{split}
$$

(We showed that Q verifies the **Bellman equation** )
***

For the next equality :
$$
\begin{split}
Q^{*}(s, a)    & = \max_{(a,\pi)} [ r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) Q^{\pi}(s', a')] \\
               & = \max_a [ r(s,a) + \gamma \max_{\pi} \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a)  Q^{\pi}(s', a')] \\
\end{split}
$$

We have that :

$$ \max_{\pi} \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a)  Q^{\pi}(s', a') \leq \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \max_{\pi} Q^{\pi}(s', a') $$

By denoting $\pi^* = argmax_{\pi} Q^{\pi}$ : 
$$ \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \max_{\pi} Q^{\pi}(s', a') = \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a)  Q^{\pi^{*}}(s', a')$$

But we also have :
$$ \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a)  Q^{\pi^{*}}(s', a') \leq \max_{\pi} \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a)  Q^{\pi}(s', a') $$ 

Thus we get that : 

$$ \max_{\pi} \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a)  Q^{\pi}(s', a') = \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \max_{\pi} Q^{\pi}(s', a') $$

So we have : 

$$
\begin{split}
Q^{*}(s, a)    & = \max_{(a,\pi)} [ r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) Q^{\pi}(s', a')] \\
               & = \max_a [ r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \max_{\pi} Q^{\pi}(s', a') ] \\
               & = \max_a [ r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \max_{(\pi,a')} Q^{\pi}(s', a') ] \\
               & = \max_a [ r(s,a) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'|s_0 = s, a_0 = a) \max_{a'} Q^{*}(s', a') ] \\
               & = r(s,a) + \gamma \mathbb{E}_{s'\sim \pi^{*}(.|s,a)}[\max_{a'} Q^{*}(s', a'))] \\
               & = \mathbb{E}_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{split}
$$
***

Since we have :

$$ 
\mathbb{E}_{s' \sim \pi^*(.|s,a)}[ r+\gamma\max_{a'}Q^*(s',a',\theta)-Q^*(s,a,\theta)]= 0
$$

Our neural network predicting the Q value will converge if we ensure this equality, thus we can minimize the L2-norm of this quantity. And that is exactly $\mathcal{L}$ proposed.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory :
            self.memory = self.memory[1:]

    def random_access(self):
        random_index = np.random.randint(len(self.memory))
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        shape = tf.keras.backend.expand_dims(s, axis=0).shape
        pred = self.model.predict(s.reshape(shape))
        return pred.argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # Sample a minibatch of transitions from the replay buffer
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            # store the states, rewards in the right arrays to be used for the SGD
            input_states[i] = s_ # the starting state
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_ # the reward associated to that (state,action) performed if terminal state
            else:
                ######## FILL IN
                pred = self.model.predict(n_s_.reshape(1, 5, 5, self.n_state))
                target_q[i, a_] = r_ + self.discount * np.max(pred) # the reward associated to that (state,action) performed if not terminal
        ######## FILL IN
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(40, activation="relu"))
#         model.add(Dense(10, activation="relu"))
        model.add(Dense(4))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(model.summary())
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 164       
Total params: 2,204
Trainable params: 2,204
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/030 | Loss 0.0197 | Win/lose count 6.0/5.0 (1.0)
Epoch 001/030 | Loss 0.0099 | Win/lose count 3.0/6.0 (-3.0)
Epoch 002/030 | Loss 0.0035 | Win/lose count 1.0/2.0 (-1.0)
Epoch 003/030 | Loss 0.0070 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/030 | Loss 0.0153 | Win/lose count 3.0/1.0 (2.0)
Epoch 005/030 | Loss 0.0075 | Win/lose count 7.0/4.0 (3.0)
Epoch 006/030 | Loss 

We see that even with this NN the mouse get stuck in some cells and keeps oscillating between some states and do not got to explore

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(40,
                         (2,2), 
                         input_shape=(5,5,self.n_state,),
                         activation='relu'))
        model.add(Conv2D(20,
                         (2,2),
                         activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(model.summary())

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 4, 4, 40)          360       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 20)          3220      
_________________________________________________________________
flatten_3 (Flatten)          (None, 180)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 724       
Total params: 4,304
Trainable params: 4,304
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/030 | Loss 0.0011 | Win/lose count 3.5/8.0 (-4.5)
Epoch 001/030 | Loss 0.0106 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/030 | Loss 0.0027 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/030 | Loss 0.0085 | Win/lose count 2.0/5.0 (-3.0)
Epoch 

I tested 3 different FC architectures that I trained for 30 epochs : 
- 1 layer of 40 neurons and a final 4 neurons layer (output of regression) : On average, the learner achieves positive rewards at each epoch leading us to think that it is indeed learning. But even in this case, the reward is not that great and is always under 10 due to the lack of exploration always. We observe the same problem of getting stuck in some states too.
- 2 layers of 10 neurons followed by relu and a final 4 neurons layer (output of regression) : Does not learn at all (very low of negative rewards)
- 4 layers of 5 neurons followed by relu and a final 4 neurons layer (output of regression) : Same as the previous one.

After that I moved on with trying with the other parameters with the "best" architecture I got and thus changed the $\epsilon$ parameter and the learning rate, it turned out that the couple (0.1 , 0.1) was a good hyperparams set since it game on aveage on all 30 epochs the best reward (win/lose count)

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 4, 40)          360       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 20)          3220      
_________________________________________________________________
flatten_4 (Flatten)          (None, 180)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 724       
Total params: 4,304
Trainable params: 4,304
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 50)                0         
_________________________

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

The CNN has a better performance than the FC network, since it looks like he is good at dodging the negative rewards but the problem for both models is that they get stuck in some actions and seem not to exit them.

The temperature's impact on the agents score is that :
- Set to a low value (0.05, or 0.1 like before), the agents do not manage to retrieve high rewarding states. 
- Set to a high value (tested with 0.8), the agents get positive average rewards. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon_decay=1,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Decrease the exploration rate
        agent.set_epsilon(agent.epsilon * epsilon_decay)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, round(win, 2), round(lose, 2), round(win-lose, 2)))
        agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

In [0]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        
    def get_frame(self, t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        # Exploration malus
        if train:
            reward = -self.malus_position[self.x, self.y] # Punish the agent if the state was visited before
        self.malus_position[self.x, self.y] = 0.1 # Mark the current state as visited
        # Normal bonus
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # The state now includes wether the surrounding states were visited before
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
    
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 4, 4, 40)          520       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 20)          3220      
_________________________________________________________________
flatten_6 (Flatten)          (None, 180)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 724       
Total params: 4,464
Trainable params: 4,464
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0152 | Win/lose count 7.0/22.3 (-15.3)
Epoch 002/030 | Loss 0.0077 | Win/lose count 2.0/18.0 (-16.0)
Epoch 003/030 | Loss 0.0122 | Win/lose count 3.5/28.5 (-25.0)
Epoch 004/030 | Loss 0.0070 | Win/lose count 7.2/22.1 (-14.

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 1.0/0. Average score (1.0)
Win/lose count 5.5/2.0. Average score (2.25)
Win/lose count 5.5/0. Average score (3.3333333333333335)
Win/lose count 7.5/2.0. Average score (3.875)
Win/lose count 4.5/0. Average score (4.0)
Win/lose count 2.5/2.0. Average score (3.4166666666666665)
Win/lose count 8.5/1.0. Average score (4.0)
Win/lose count 2.5/0. Average score (3.8125)
Win/lose count 6.0/0. Average score (4.055555555555555)
Win/lose count 6.0/1.0. Average score (4.15)
Win/lose count 1.5/0. Average score (3.909090909090909)
Win/lose count 7.0/1.0. Average score (4.083333333333333)
Win/lose count 11.5/2.0. Average score (4.5)
Win/lose count 2.0/0. Average score (4.321428571428571)
Win/lose count 6.0/1.0. Average score (4.366666666666666)
Win/lose count 13.5/2.0. Average score (4.8125)
Win/lose count 1.0/0. Average score (4.588235294117647)
Win/lose count 5.5/2.0. Average score (4.527777777777778)
Win/lose count 6.5/0. Average score (4.631578947368421)
Win/lose count 1.0/0. Averag

Using the two proposed modifications for the traning helped increase exploration of the environment, and thus led to improving the rewards collected (From 1 on average to 4 on average). But what is weird here is that it seems that it does not learn in the training phase (negative rewards always) and I expected more average reward in the test phase.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***